# Fetching codes

In [ ]:
!git clone https://github.com/minhnguyent546/machine-translation-en-vi.git
%cd machine-translation-en-vi

In [ ]:
!pip install -q datasets pyvi

# Configuration

In [ ]:
from config import get_config
from pathlib import Path
import os

storage_dir = '/kaggle/working/storage'
Path(storage_dir).mkdir(parents=True, exist_ok=True)

config = get_config()
config['checkpoints_dir'] = f'{storage_dir}/checkpoints'
config['experiment_name'] = f'{storage_dir}/runs/model'
config['batch_size'] = 32
config['num_epochs'] = 10
config['seq_length'] = 120
config['num_validation_samples'] = 200 # -1 for evaluating the whole dataset
config['num_test_samples'] = -1
config['beam_size'] = 5
# config['preload'] = None

# Preprocessing

In [ ]:
from preprocess import preprocess
preprocess(config)

# Tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
! ! command -v ngrok &>/dev/null && \
        wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz && \
        tar -xvzf ngrok-v3-stable-linux-amd64.tgz -C /usr/local/bin

!ngrok authtoken <YOUR_NGROK_AUTH_TOKEN>

import os
import multiprocessing
pool = multiprocessing.Pool(processes = 10)
results_of_processes = [pool.apply_async(os.system, args=(cmd,))
                        for cmd in [
                            f'tensorboard --logdir {config["experiment_name"]} --host 0.0.0.0 --port 6006 &',
                            "/usr/local/bin/ngrok http 6006 &"
                        ]]

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard running at:', json.load(sys.stdin)['tunnels'][0]['public_url'])"

# Training

In [ ]:
from train_model import train_model
train_model(config)

# Testing

In [ ]:
from test_model import test_model
test_model(config)